In [ ]:
# 자료출처
# https://butter-shower.tistory.com/140

# 다양한 데이터 전처리 기법 - 결측치, 중복, 이상치, 정규화, 원-핫 인코딩, 구간화(binning)
# https://butter-shower.tistory.com/144?category=737935#%EC%A4%91%EB%B3%B5%EB%90%9C-%EB%8D%B0%EC%9D%B4%ED%84%B0

In [191]:
import pandas as pd
import matplotlib.pyplot as plt  # 그래프 그리기

from IPython.core.display import display, HTML
from konlpy.tag import Okt,Kkma,Hannanum,Komoran,Mecab   


okt = Okt()
kkma = Kkma()
hannanum = Hannanum()
komoran = Komoran
mecab = Mecab # 윈도우에서 지원되지 않는다.

display(HTML("<style>.container {width:90% !important;}</style>"))

# 1. 데이터 불러오기

In [352]:
df = pd.read_csv('E:/[python]/5. [python] 자연어/20200101_contents.csv')

In [356]:
print(df.info())

# 칼럼별 결측치 개수 = len(trade) - trade.count()
print(df.isnull().sum())

print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17628 entries, 0 to 17627
Data columns (total 16 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   no            17628 non-null  int64 
 1   date          17628 non-null  int64 
 2   year          17628 non-null  int64 
 3   month         17628 non-null  int64 
 4   day           17628 non-null  int64 
 5   wday          17628 non-null  object
 6   edittime      17628 non-null  object
 7   url           17628 non-null  object
 8   main          17628 non-null  object
 9   sub           17628 non-null  object
 10  press         17628 non-null  object
 11  reporter      16019 non-null  object
 12  email         12941 non-null  object
 13  title         17628 non-null  object
 14  contents      17604 non-null  object
 15  review_count  17627 non-null  object
dtypes: int64(5), object(11)
memory usage: 2.2+ MB
None
no                 0
date               0
year               0
month           

# 2. 데이터 전처리

In [196]:
# 한글 이외 제거
df['contents'] = df['contents'].str.replace("[^ㄱ-ㅎ ㅏ-ㅣ 가-힣]","")

# 중복제거
df.drop_duplicates(subset=['contents'], inplace = True)
print('뉴스 기사의 개수 : ', len(df))

# NULL값을 갖는 행 제거.
df.dropna(axis=0,inplace=True)
print(df.isnull().sum())
# trade.dropna(how='all', subset = ['wday', 'day', 'title'], inplace = True)
# subset : 특정 칼럼을 선택
# how : 선택한 칼럼 전부가 결측치인 행을 삭제하겠다는 의미. ('any' : 하나라도 결측치인 경우에 선택)
# inplace : True면 변경된 값이 해당 DataFrame에 바로 적용

df['main'].value_counts().plot(kind = 'bar')

# 3. 토큰화 및 불용어 제거

In [ ]:
stopwords = ['에','는','은','을','했','에게','있','이','의','하','한','다','과','때문','할','수','무단','따른','및','금지','전재','경향신문','기자','는데','가','등','들','파이낸셜','저작','등','뉴스']

In [234]:
from tqdm import tqdm_notebook
def preprocessing(data):
    text_data = []
    
    for sentence in tqdm_notebook(data):
        temp_data = []
        # 토큰화
        temp_data = okt.morphs(sentence)
        # 불용어 제거
        temp_data = [word for word in temp_data if not word in stopwords]
        text_data.append(temp_data)
    
    text_data = list(map(' '.join, text_data))
    
    return text_data

In [236]:
text_data = preprocessing(df['contents'])
print(text_data[0])


이미지 출처 연합뉴스 아시아 경제 유병돈 전북 전주시 노송동 얼굴 없는 천사 놓고 간 성금 훔친 대 명 구속 됐다 전주 지법 최정윤 판사 일 특수 절도 혐의 를 받는 씨 와 씨 대한 구속 전 피의자 심문 영장 실질 실사 에서 도주 우려 있다면서 구속영장 발부 했다 씨 지난달 일 오전 시 께 전주 노송동 주민 센터 뒤편 에서 얼굴 없는 천사 로 불리는 익명 기 부자 두고 간 기 부금 여 만원 훔쳐 달아난 혐의 를 받는다 경찰 따르면 얼굴 없는 천사 몰래 기부 이번 에도 이뤄질 것 으로 예상 하고 범행 모의 했다 경찰 조사 결과 씨 지난달 일 부터 주민 센터 인근 스포츠 유틸리티 차 를 세워놓고 얼굴 없는 천사 성금 놓고 가기 를 기다린 것 으로 드러났다 차량 번호판 에는 물 묻힌 휴지 를 붙여 식별 어렵게 했다 경찰 평소 동네 에서 눈 띄지 않던 를 수상하게 여긴 주민 제보 로 범행 시간 여 만에 충남 논산 대전 인근 에서 씨 와 씨 를 각각 붙잡았다 훔친 성금 만원 도 되찾았다 한편 얼굴 없는 천사 년 부터 지난해 까지 해도 거르 지 않고 주민 센터 인근 수천만 억원 상당 성금 놓고 갔다 그러면서도 자신 이름 얼굴 단 차례 도 공개 하지 않았다 유병돈


# 4. 머신러닝 사용하기

In [323]:
# 사이킷런(scikit-learn 또는 sklearn)는 훈련 데이터와 테스트 데이터를 분리하는데 유용한 train_test_split() 함수를 제공합니다. 
# train_test_split()에 사용할 데이터를 입력하면 훈련 데이터와 테스트 데이터로 분리해줍니다.

from sklearn.model_selection import train_test_split # train, test set 나누기 위함.
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [346]:
X_train, X_test, y_train, y_test = train_test_split(text_data,df['main'],random_state=0)
print("훈련용 뉴스 기사의 개수 : ", len(X_train))
print("테스트용 뉴스 기사의 개수 : ", len(X_test))
print("훈련용 레이블의 개수 : ", len(y_train))
print("테스트용 레이블의 개수 : ", len(y_test))

훈련용 뉴스 기사의 개수 :  6861
테스트용 뉴스 기사의 개수 :  2287
훈련용 레이블의 개수 :  6861
테스트용 레이블의 개수 :  2287


In [347]:
# fit_transform() 함수는 fit와 transform을 연이어 수행하는 함수
# CountVectorizer.fit_transform()은 단어 데이터를 학습하고 문서 데이터를 document-form matrix로 변환하는 두가지 작업을 해준다.

# 단어의 수를 카운트하는 사이킷런의 카운트벡터라이저
count_vec = CountVectorizer()
X_train_counts = count_vec.fit_transform(X_train)

# 카운트벡터라이저의 결과로부터 TF-IDF 결과를 얻습니다.
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

# 나이브베이즈 분류기를 수행ㅎ압니다.
# X_train은 TF-IDF 벡터, y_train은 레이블입니다.
clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [348]:
# 나이브 베이즈 분류기가 학습되었습니다. 모델이 학습되었다면 그 다음 해야 할 일은 바로 '테스트'입니다. 
# 이 모델이 제대로 학습되었는지를 확인해봐야겠죠?
# 텍스트를 입력하면 자동으로 TF-IDF 벡터로 바꾸는 전처리 함수를 만들어보겠습니다. 
# 이 함수를 통해 텍스트를 바로 나이브베이즈 분류기의 입력으로 사용함으로써 보다 용이하게 테스트 할 수 있습니다.

def tfidf_vectorizer(data):
    data_counts = count_vec.transform(data)
    data_tfidf = tfidf_transformer.transform(data_counts)
    return data_tfidf

In [350]:
# 임의의 뉴스에 대해서 카테고리를 확인해봅시다. 
# 이 뉴스들은 훈련 데이터, 테스트 데이터 그 어디에도 속하지 않는 임의로 가져온 뉴스입니다. 
# clf.predict()는 임의의 입력에 대해서 나이브 베이즈 분류기가 예측한 값을 리턴합니다.

new_sent = preprocessing(["민주당 일각에서 법사위의 체계·자구 심사 기능을 없애야 한다는 \
                           주장이 나오는 데 대해 “체계·자구 심사가 법안 지연의 수단으로 \
                          쓰이는 것은 바람직하지 않다”면서도 “국회를 통과하는 법안 중 위헌\
                          법률이 1년에 10건 넘게 나온다. 그런데 체계·자구 심사까지 없애면 매우 위험하다”고 반박했다."])
print(clf.predict(tfidf_vectorizer(new_sent)))


['사회']


In [349]:
# 머신러닝 모델의 성능 측정 방법 중 하나인 F1-Score에서 81%의 준수한 정확도를 얻어냈습니다!
# F1 스코어에 대해서는 아래 링크에서 확인해주세요!
# F1 스코어 : http://seb.kr/w/F1_%EC%8A%A4%EC%BD%94%EC%96%B4

y_pred = clf.predict(tfidf_vectorizer(X_test))
print(metrics.classification_report(y_test, y_pred))

              precision    recall  f1-score   support

          IT       0.00      0.00      0.00        57
          경제       0.78      0.82      0.80       317
          국제       0.94      0.27      0.42       124
          문화       0.00      0.00      0.00        65
          사회       0.71      0.94      0.81       517
         스포츠       0.99      0.95      0.97       296
          연예       0.93      0.99      0.96       583
          정치       0.83      0.87      0.85       314
          칼럼       0.00      0.00      0.00        14

    accuracy                           0.83      2287
   macro avg       0.57      0.54      0.53      2287
weighted avg       0.80      0.83      0.80      2287

